# Replace exchange-correlation by PBE and rVV10

In [1]:
import qepy
import importlib

In [2]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [3]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
import matplotlib.pyplot as plt

In [4]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'"
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'",
        'input_dft' : "'PBE'"
    },
    '&electrons': {
        'conv_thr' : 1e-8
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al_ONCV_PBE-1.2.upf'],
    'k_points automatic': ['4 4 4 0 0 0'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

In [5]:
driver=Driver(qe_options=qe_options, iterative = True, logfile=True)

In [6]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.get_energy()

Iter:  0  - Conv:  0.07520824059877651
Iter:  1  - Conv:  0.0013949184647052578
Iter:  2  - Conv:  3.710927074024824e-05
Iter:  3  - Conv:  1.32989451870516e-07
Iter:  4  - Conv:  4.295314300787336e-10


-137.91449181100646

In [7]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp2.out')

In [8]:
from dftpy.functional import XC
def eval_xc(driver, xc):
    rho = driver.get_density()
    field = driver.data2field(rho)
    func = xc(field)
    pot = driver.field2data(func.potential)*2
    energy = func.energy*2
    return pot, energy

In [9]:
xc = XC('PBE')
for i in range(60):
    extpot, ex = eval_xc(driver, xc)
    driver.set_external_potential(potential=extpot*1.0, extene=ex, exttype=('xc'))
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break

driver.get_energy()

Iter:  0  - Conv:  0.07505707163989339
Iter:  1  - Conv:  0.0014061390737141637
Iter:  2  - Conv:  3.633193873511027e-05
Iter:  3  - Conv:  1.4088993347162226e-07
Iter:  4  - Conv:  1.0588484593621504e-09


-137.91448745432706

In [10]:
xc = XC('RVV10')
for i in range(60):
    extpot, ex = eval_xc(driver, xc)
    driver.set_external_potential(potential=extpot, extene=ex, exttype=('xc'))
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break

driver.get_energy()

Iter:  0  - Conv:  1.777016139224226e-05
Iter:  1  - Conv:  1.119653777264494e-06
Iter:  2  - Conv:  1.4327807862798431e-08
Iter:  3  - Conv:  1.125385082151563e-10


-138.0116925869026